In [4]:
import struct
import pyodbc
from azure.identity import AzureCliCredential  # or InteractiveBrowserCredential, etc.

# --- fill these from Fabric connection strings ---
SERVER   = "tcp:dwqurwscuuxern3c5uxx3n3hgq-apvmkjmaa6xe3knachpdztfq3e.datawarehouse.fabric.microsoft.com,1433"
DATABASE = "lh_aura_001"  # the SQL endpoint's DB (e.g., lakehouse name)

# Get an AAD access token for SQL (resource = database.windows.net)
# Tip: 'az login' in the same VS Code shell makes AzureCliCredential work immediately.
scope = "https://database.windows.net/.default"
token = AzureCliCredential().get_token(scope).token

# ODBC expects the token as UTF-16-LE with a 4-byte length prefix (SQL_COPT_SS_ACCESS_TOKEN = 1256)
token_bytes  = token.encode("utf-16-le")
token_struct = struct.pack("=i", len(token_bytes)) + token_bytes

conn_str = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    "Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;"
)

with pyodbc.connect(conn_str, attrs_before={1256: token_struct}) as conn:
    rows = conn.cursor().execute("SELECT TOP (5) * FROM dbo.iris;").fetchall()
    for r in rows:
        print(r)


(5.1, 3.5, 1.4, 0.2, 'Iris-setosa')
(4.9, 3.0, 1.4, 0.2, 'Iris-setosa')
(4.7, 3.2, 1.3, 0.2, 'Iris-setosa')
(4.6, 3.1, 1.5, 0.2, 'Iris-setosa')
(5.0, 3.6, 1.4, 0.2, 'Iris-setosa')


In [5]:
rows = conn.cursor().execute("SELECT TOP (5) * FROM dbo.iris;").fetchall()
for r in rows:
    print(r)

(5.1, 3.5, 1.4, 0.2, 'Iris-setosa')
(4.9, 3.0, 1.4, 0.2, 'Iris-setosa')
(4.7, 3.2, 1.3, 0.2, 'Iris-setosa')
(4.6, 3.1, 1.5, 0.2, 'Iris-setosa')
(5.0, 3.6, 1.4, 0.2, 'Iris-setosa')
